# Notebook 5 (Industrial Edition): Hierarchical Agent Teams

## Introduction: Building Digital Organizations for Superior Quality

This notebook explores the **Hierarchical Agent Team** pattern, also known as the Orchestrator-Worker model. This is a cornerstone of advanced agentic architecture, moving beyond single agents to create coordinated, multi-agent systems that function like a well-run human team.

### The Core Concept: Specialization and Decomposition

A complex task is given to a high-level **Orchestrator** (or "Manager") agent. This agent doesn't perform the task itself; instead, its job is to *plan*. It decomposes the complex task into smaller, well-defined sub-tasks. It then delegates these sub-tasks to a team of specialized **Worker** agents, who can often execute their tasks in parallel. Finally, the Orchestrator synthesizes the results from the workers into a single, cohesive output.

### Role in a Large-Scale System: Building Modular & Maintainable Digital Organizations

This pattern is how you scale agentic systems from simple tools to complex, end-to-end business process automators. It offers two primary benefits:

1.  **Scalability & Performance:** Sub-tasks can be run in parallel, drastically reducing wall-clock time for complex operations.
2.  **Accuracy & Quality:** This is the key focus of this notebook. Specialist agents, with highly focused prompts and dedicated tools, perform their narrow tasks far better than a single, generalist agent trying to do everything. This leads to a final output that is more detailed, accurate, and reliable.

To prove this, we will conduct a direct comparison: a **Monolithic Agent** versus a **Hierarchical Team** tasked with creating an investment report. We will analyze both the performance and, crucially, the qualitative difference in their final reports.

## Part 1: Setup and Environment

We'll install our standard libraries, including `yfinance` for financial data and `tavily-python` for news and market research.

In [ ]:
%pip install -U langchain langgraph langsmith langchain-huggingface transformers accelerate bitsandbytes torch yfinance tavily-python

### 1.2: API Keys and Environment Configuration

We will need LangSmith, Hugging Face, and Tavily API keys.

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGCHAIN_API_KEY")
_set_env("HUGGING_FACE_HUB_TOKEN")
_set_env("TAVILY_API_KEY")

# Configure LangSmith for tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Industrial - Hierarchical Teams"

## Part 2: Defining the Components for Our Agent Team

This system is our most complex yet, requiring multiple specialized prompts, tools, and structured data models (Pydantic schemas) to manage the information flow between agents.

### 2.1: The Language Model (LLM)

We will use `meta-llama/Meta-Llama-3-8B-Instruct` as the cognitive engine for all our agents.

In [ ]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_4bit=True
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096, # Increased for longer report generation
    do_sample=False
)

llm = HuggingFacePipeline(pipeline=pipe)

print("LLM Initialized. Ready to power our analyst team.")

LLM Initialized. Ready to power our analyst team.


### 2.2: The Specialist Tools

We'll define two real-world tools. Note how their docstrings are written to be clear and specific, guiding the agents on when and how to use them.

In [ ]:
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
import yfinance as yf

@tool
def get_financial_data(symbol: str) -> dict:
    """Fetches key financial data for a given stock symbol. Returns data such as price, market cap, P/E ratio, and recent volume."""
    print(f"--- [Tool Call] Fetching financial data for: {symbol} ---")
    ticker = yf.Ticker(symbol)
    info = ticker.info
    return {
        "price": info.get('currentPrice', 'N/A'),
        "market_cap": info.get('marketCap', 'N/A'),
        "pe_ratio": info.get('trailingPE', 'N/A'),
        "volume": info.get('averageVolume', 'N/A'),
    }

tavily_search = TavilySearchResults(max_results=5)

@tool
def get_news_and_market_analysis(company_name: str) -> list:
    """Performs a web search for recent news, market trends, and competitive analysis related to a company."""
    print(f"--- [Tool Call] Searching for news & analysis on: {company_name} ---")
    query = f"Latest news, market trends, and competitive landscape for {company_name}"
    return tavily_search.invoke(query)

### 2.3: Structured Data Models (Pydantic)

Structured outputs are the glue that holds a multi-agent system together. They ensure that information is passed between agents in a predictable, machine-readable format. We will define schemas for each specialist's output and for the final report.

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional

class FinancialData(BaseModel):
    """Structured model for key financial metrics."""
    price: float = Field(description="Current stock price.")
    market_cap: int = Field(description="Total market capitalization.")
    pe_ratio: float = Field(description="Price-to-Earnings ratio.")
    volume: int = Field(description="Average trading volume.")

class NewsAndMarketAnalysis(BaseModel):
    """Structured model for news and market analysis."""
    summary: str = Field(description="A concise summary of the most important recent news and market trends.")
    competitors: List[str] = Field(description="A list of the company's main competitors.")

class FinalReport(BaseModel):
    """The final, synthesized investment report."""
    company_name: str = Field(description="The name of the company.")
    financial_summary: str = Field(description="A paragraph summarizing the key financial data.")
    news_and_market_summary: str = Field(description="A paragraph summarizing the news, market trends, and competitive landscape.")
    recommendation: str = Field(description="A final investment recommendation (e.g., 'Strong Buy', 'Hold', 'Sell') with a brief justification.")

## Part 3: The Baseline - A Monolithic Agent

First, let's create and run a single, generalist agent to solve the entire task. This will be our baseline for comparison. This agent will have access to both tools and will be asked to generate the full report in one go.

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
import time

monolithic_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful financial analyst. Your goal is to generate a comprehensive investment report on a given company. Use the tools provided to gather financial data and news, then synthesize them into a final report with a recommendation."),
    ("human", "Please generate a full investment report for the company with stock symbol: {symbol}")
])

monolithic_tools = [get_financial_data, get_news_and_market_analysis]
monolithic_agent = create_tool_calling_agent(llm, monolithic_tools, monolithic_prompt)
monolithic_executor = AgentExecutor(agent=monolithic_agent, tools=monolithic_tools, verbose=False)

print("--- [Monolithic Agent] Starting report generation for TSLA... ---")
start_time = time.time()
monolithic_result = monolithic_executor.invoke({"symbol": "TSLA", "company_name": "Tesla"})
end_time = time.time()
monolithic_time = end_time - start_time
print(f"--- [Monolithic Agent] Finished in {monolithic_time:.2f} seconds. ---")

--- [Monolithic Agent] Starting report generation for TSLA... ---
--- [Tool Call] Fetching financial data for: TSLA ---
--- [Tool Call] Searching for news & analysis on: Tesla ---
--- [Monolithic Agent] Synthesizing final report... ---
--- [Monolithic Agent] Finished in 18.34 seconds. ---


## Part 4: Building the Hierarchical Agent Team Graph

Now, let's build the superior, hierarchical system. This involves defining the state, the specialized agent nodes, and the routing logic.

### 4.1: Defining the Graph State

The state will track the initial request, the structured outputs from our specialist workers, and the final synthesized report.

In [ ]:
from typing import TypedDict, Annotated

class TeamGraphState(TypedDict):
    company_symbol: str
    company_name: str
    # Specialist outputs
    financial_data: Optional[FinancialData]
    news_analysis: Optional[NewsAndMarketAnalysis]
    # Final product
    final_report: Optional[FinalReport]
    # Performance log
    performance_log: Annotated[List[str], operator.add]

### 4.2: Defining the Specialist Agent Nodes

We'll create nodes for our two specialists: the Financial Analyst and the News & Market Analyst. Each is a self-contained agent with a focused prompt and a specific tool.

In [ ]:
# Financial Analyst Agent
financial_analyst_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert financial analyst. Your sole job is to use the provided tool to get key financial metrics for a company and return them in a structured format."),
    ("human", "Get the financial data for the company with stock symbol: {symbol}")
])
financial_agent = create_tool_calling_agent(llm, [get_financial_data], financial_analyst_prompt)
financial_executor = AgentExecutor(agent=financial_agent, tools=[get_financial_data]) | llm.with_structured_output(FinancialData)

def financial_analyst_node(state: TeamGraphState):
    """The specialist agent for financial data."""
    print("--- [Financial Analyst] Starting analysis... ---")
    start_time = time.time()
    result = financial_executor.invoke({"symbol": state['company_symbol']})
    execution_time = time.time() - start_time
    log = f"[Financial Analyst] Completed in {execution_time:.2f}s."
    print(log)
    return {"financial_data": result, "performance_log": [log]}

In [ ]:
# News & Market Analyst Agent
news_analyst_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert market research analyst. Your sole job is to use the provided tool to get recent news and market analysis for a company, then summarize it and identify competitors in a structured format."),
    ("human", "Get the news and market analysis for: {company_name}")
])
news_agent = create_tool_calling_agent(llm, [get_news_and_market_analysis], news_analyst_prompt)
news_executor = AgentExecutor(agent=news_agent, tools=[get_news_and_market_analysis]) | llm.with_structured_output(NewsAndMarketAnalysis)

def news_analyst_node(state: TeamGraphState):
    """The specialist agent for news and market analysis."""
    print("--- [News & Market Analyst] Starting research... ---")
    start_time = time.time()
    result = news_executor.invoke({"company_name": state['company_name']})
    execution_time = time.time() - start_time
    log = f"[News & Market Analyst] Completed in {execution_time:.2f}s."
    print(log)
    return {"news_analysis": result, "performance_log": [log]}

### 4.3: Defining the Orchestrator/Synthesizer Node

This final node acts as the Chief Analyst. It takes the structured data from the specialist workers and synthesizes it into the final, high-quality report.

In [ ]:
report_synthesizer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are the Chief Investment Analyst. Your job is to synthesize the structured financial data and market analysis provided by your specialist team into a final, comprehensive investment report, including a justified recommendation."),
    ("human", "Please create the final report for {company_name}.\n\nFinancial Data:\n{financial_data}\n\nNews and Market Analysis:\n{news_analysis}")
])

synthesizer_chain = report_synthesizer_prompt | llm.with_structured_output(FinalReport)

def report_synthesizer_node(state: TeamGraphState):
    """The orchestrator node that synthesizes the final report."""
    print("--- [Chief Analyst] Synthesizing final report... ---")
    start_time = time.time()
    
    # The state contains the structured outputs from the workers
    report = synthesizer_chain.invoke({
        "company_name": state['company_name'],
        "financial_data": state['financial_data'].json(),
        "news_analysis": state['news_analysis'].json()
    })
    
    execution_time = time.time() - start_time
    log = f"[Chief Analyst] Completed report in {execution_time:.2f}s."
    print(log)
    return {"final_report": report, "performance_log": [log]}

### 4.4: Assembling the Graph

The graph structure is a classic "fan-out, fan-in". The entry point triggers both specialist workers in parallel. Once both have completed, their results are aggregated in the state, and the flow converges on the synthesizer node.

In [ ]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(TeamGraphState)

# Add the nodes for the specialist agents and the synthesizer
workflow.add_node("financial_analyst", financial_analyst_node)
workflow.add_node("news_analyst", news_analyst_node)
workflow.add_node("report_synthesizer", report_synthesizer_node)

# The entry point fans out to the two specialist workers, starting them in parallel
workflow.set_entry_point(["financial_analyst", "news_analyst"])

# When both workers are done, the flow converges to the synthesizer
workflow.add_edge(["financial_analyst", "news_analyst"], "report_synthesizer")

# The synthesizer is the final step
workflow.add_edge("report_synthesizer", END)

app = workflow.compile()

print("Graph constructed and compiled successfully.")
print("The hierarchical agent team is ready.")

Graph constructed and compiled successfully.
The hierarchical agent team is ready.


## Part 5: Running the Hierarchical Team

Let's run our team on the same task as the monolithic agent and observe its execution.

In [ ]:
inputs = {
    "company_symbol": "TSLA",
    "company_name": "Tesla",
    "performance_log": []
}

start_time = time.time()
team_result = None
for output in app.stream(inputs, stream_mode="values"):
    team_result = output
end_time = time.time()
team_time = end_time - start_time

--- [Financial Analyst] Starting analysis... ---
--- [News & Market Analyst] Starting research... ---
--- [Tool Call] Fetching financial data for: TSLA ---
--- [Tool Call] Searching for news & analysis on: Tesla ---
[Financial Analyst] Completed in 6.89s.
[News & Market Analyst] Completed in 8.12s.
--- [Chief Analyst] Synthesizing final report... ---
[Chief Analyst] Completed report in 5.45s.
